In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

import tensorflow as tf
import tensorflow_hub as hub
import random

# DATA PREPARATION

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train, X_test = X_train/255, X_test/255

In [3]:
class_order = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Create a list to store the ordered training items
ordered_train_images = []
ordered_train_labels = []

# Iterate over the class order and extract the corresponding training items
for i in range(len(class_order)):
    class_name = class_order[i]
    class_index = i
    class_indices = np.where(y_train == class_index)[0]
    class_images = X_train[class_indices]
    class_labels = y_train[class_indices]
    ordered_train_images.extend(class_images)
    ordered_train_labels.extend(class_labels)
# Convert the ordered training items back to numpy arrays
X_train = np.array(ordered_train_images)
y_train = np.array(ordered_train_labels)

In [4]:
X_train = np.concatenate([X_train[:1000], X_train[5000:6000],
                          X_train[10000:11000], X_train[15000:16000],
                          X_train[20000:21000], X_train[25000:26000],
                          X_train[30000:31000], X_train[35000:36000],
                          X_train[40000:41000], X_train[45000:46000]])
y_train = np.concatenate([y_train[:1000], y_train[5000:6000],
                          y_train[10000:11000], y_train[15000:16000],
                          y_train[20000:21000], y_train[25000:26000],
                          y_train[30000:31000], y_train[35000:36000],
                          y_train[40000:41000], y_train[45000:46000]])

# step2

In [5]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

In [6]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

style_images = [load_image("4.jpeg"), load_image("frida.jpg"), load_image("1.jpg")]

In [7]:
content_images = []

for i in range(20):
    content = tf.image.convert_image_dtype(X_train[i], tf.float32)
    content = content[tf.newaxis, :]
    content_images.append(content)

In [8]:
def generate_samples(content_images, style_images, style_transfer_model):
    
    positive_samples = []
    negative_samples = []
    
    # Generate positive samples
    for content in content_images:
        for style in style_images:
            stylized_image = style_transfer_model(tf.constant(content), tf.constant(style))[0]
            positive_samples.append((content, stylized_image))
    
    # Generate negative samples
    for content1 in content_images:
        for content2 in content_images:
            if  not (np.array_equal(content1, content2)):
                random_style = style_images[np.random.randint(0, len(style_images)-1)]
                stylized_image = style_transfer_model(tf.constant(content1), tf.constant(random_style))[0]
                negative_samples.append((content1, stylized_image))
    
    return positive_samples, negative_samples

In [28]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the contrastive model architecture
def create_contrastive_model(input_shape_content, input_shape_style, embedding_dim):
    input_content = tf.keras.Input(shape=input_shape_content)
    input_style = tf.keras.Input(shape=input_shape_style)
    
    # Content branch
    x_content = layers.Conv2D(32, (3, 3), activation='relu')(input_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Conv2D(64, (3, 3), activation='relu')(x_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Flatten()(x_content)
    x_content = layers.Dense(embedding_dim, activation='relu')(x_content)
    
    # Style branch
    x_style = layers.Conv2D(32, (3, 3), activation='relu')(input_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Conv2D(64, (3, 3), activation='relu')(x_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Flatten()(x_style)
    x_style = layers.Dense(embedding_dim, activation='relu')(x_style)
    
    # Concatenate content and style embeddings
    concatenated = layers.Concatenate()([x_content, x_style])
    
    flattened = layers.Flatten()(concatenated)
    # Normalize the concatenated embeddings
    normalized = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(concatenated)
    
    # Create the model with inputs and outputs
    model = tf.keras.Model(inputs=[input_content, input_style], outputs=normalized)
    return model


def contrastive_loss(y_true, y_pred, margin=1.0):
    square_pred = tf.square(1 - y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

In [29]:
positive_samples, negative_samples = generate_samples(content_images, style_images, model)

In [30]:
contrastive_model = create_contrastive_model(input_shape_content=(32, 32, 3),
                                             input_shape_style=(32, 32, 3),
                                             embedding_dim=128)

In [31]:
positive_samples = np.array(positive_samples)
negative_samples = np.array(negative_samples)

In [32]:
print(f"p:{len(positive_samples)} | n: {len(negative_samples)}")

p:60 | n: 380


In [33]:
content_images_positive = positive_samples[:, 0]
stylized_images_positive = positive_samples[:, 1]
content_images_negative = negative_samples[:, 0]
stylized_images_negative = negative_samples[:, 1]

In [34]:
positive_labels = np.ones(len(positive_samples))
negative_labels = np.zeros(len(negative_samples))

In [35]:
train_images_content = np.concatenate((content_images_positive, content_images_negative))
train_images_style = np.concatenate((stylized_images_positive, stylized_images_negative))
train_labels_content = np.concatenate((positive_labels, negative_labels))

train_images_content = np.squeeze(train_images_content, axis=1)
train_images_style = np.squeeze(train_images_style, axis=1)
# Shuffle the data
indices = np.random.permutation(len(train_images_content))
train_images_content = train_images_content[indices]
train_images_style = train_images_style[indices]
train_labels_content = train_labels_content[indices]

# Prepare the training data
train_data = ([train_images_content, train_images_style], train_labels_content)

# Split the train_data into content and style inputs
train_content_images = train_data[0][0]
train_style_images = train_data[0][1]
train_labels = train_data[1]


In [36]:
contrastive_model.compile(optimizer='adam', loss=contrastive_loss)
contrastive_model.fit([train_content_images, train_style_images], train_labels, epochs=10, batch_size=32)

Epoch 1/10
14/14 [==============================] - 1s 24ms/step - loss: 0.9002
Epoch 2/10
14/14 [==============================] - 0s 22ms/step - loss: 0.8912
Epoch 3/10
14/14 [==============================] - 0s 27ms/step - loss: 0.8902
Epoch 4/10
14/14 [==============================] - 0s 26ms/step - loss: 0.8900
Epoch 5/10
14/14 [==============================] - 0s 24ms/step - loss: 0.8899
Epoch 6/10
14/14 [==============================] - 0s 30ms/step - loss: 0.8898
Epoch 7/10
14/14 [==============================] - 1s 47ms/step - loss: 0.8898
Epoch 8/10
14/14 [==============================] - 0s 32ms/step - loss: 0.8898
Epoch 9/10
14/14 [==============================] - 0s 29ms/step - loss: 0.8897
Epoch 10/10
14/14 [==============================] - 0s 27ms/step - loss: 0.8897


In [38]:
contrastive_model.weights[0].shape

TensorShape([3, 3, 3, 32])

In [50]:
from tensorflow.keras.models import Model

# Freeze the encoder model
contrastive_model.trainable = False

# Add a new classifier layer
output_layer = tf.keras.layers.Dense(10, activation='softmax')(contrastive_model.output)

# Define the new model for classification
classification_model = Model(contrastive_model.input, output_layer)


classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the classification model
classification_model.fit(train_data[0], train_data[1], epochs=10)



Epoch 1/10
14/14 [==============================] - 1s 8ms/step - loss: 2.1836 - accuracy: 0.1364
Epoch 2/10
14/14 [==============================] - 1s 45ms/step - loss: 1.8678 - accuracy: 0.6886
Epoch 3/10
14/14 [==============================] - 0s 22ms/step - loss: 1.5824 - accuracy: 0.8636
Epoch 4/10
14/14 [==============================] - 0s 10ms/step - loss: 1.3393 - accuracy: 0.8636
Epoch 5/10
14/14 [==============================] - 0s 12ms/step - loss: 1.1355 - accuracy: 0.8636
Epoch 6/10
14/14 [==============================] - 0s 9ms/step - loss: 0.9771 - accuracy: 0.8636
Epoch 7/10
14/14 [==============================] - 0s 12ms/step - loss: 0.8551 - accuracy: 0.8636
Epoch 8/10
14/14 [==============================] - 0s 7ms/step - loss: 0.7633 - accuracy: 0.8636
Epoch 9/10
14/14 [==============================] - 0s 9ms/step - loss: 0.6963 - accuracy: 0.8636
Epoch 10/10
14/14 [==============================] - 0s 7ms/step - loss: 0.6455 - accuracy: 0.8636


import tensorflow as tf
from tensorflow.keras import layers

# Define the contrastive model architecture
def create_contrastive_model(input_shape, embedding_dim):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(embedding_dim, activation='relu')(x)
    outputs = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Generate positive and negative samples
positive_samples, negative_samples = generate_samples(a, style_images, model)

# Combine positive and negative samples
all_samples = positive_samples + negative_samples
labels = [1] * len(positive_samples) + [0] * len(negative_samples)

# Shuffle the samples and labels in the same order
combined = list(zip(all_samples, labels))
random.shuffle(combined)
all_samples, labels = zip(*combined)

# Convert samples and labels to TensorFlow tensors
all_samples = tf.convert_to_tensor(all_samples)
labels = tf.convert_to_tensor(labels)

# Split the data into training and validation sets
train_size = int(0.8 * len(all_samples))
train_samples, val_samples = all_samples[:train_size], all_samples[train_size:]
train_labels, val_labels = labels[:train_size], labels[train_size:]

# Reshape the samples to remove the extra dimension
train_samples = tf.reshape(train_samples, (-1,) + train_samples.shape[2:])
val_samples = tf.reshape(val_samples, (-1,) + val_samples.shape[2:])

# Create the contrastive model
input_shape = (1,32, 32, 3)  # Modify according to your image shape
embedding_dim = 128  # Modify according to your desired embedding dimension
contrastive_model = create_contrastive_model(input_shape, embedding_dim)

# Define the contrastive loss function
def contrastive_loss(y_true, y_pred, margin=1.0):
    square_pred = tf.square(1 - y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

# Compile the model
contrastive_model.compile(optimizer='adam', loss=contrastive_loss)

# Train the contrastive model
contrastive_model.fit(train_samples, train_labels, validation_data=(val_samples, val_labels), epochs=10)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Define base model for feature extraction (MobilenetV2 is used here)
base_model = tf.keras.applications.MobileNetV2(
    include_top=False, weights='imagenet', input_shape=(32, 32, 3), pooling='avg'
)

# Add a new projection head for contrastive learning
projection_head = Dense(128, activation='relu')(base_model.output)

# Define the projection model
projection_model = Model(base_model.input, projection_head)

# Define the encoder model
encoder_model = Model(base_model.input, base_model.output)

# Define the contrastive learning model
contrastive_model = Model(base_model.input, projection_head)

def contrastive_loss(y_true, embeddings_normalized, embeddings_projected):
    # Expand the dimensions of y_true
    y_true_expanded = tf.tile(tf.expand_dims(1 - y_true, axis=1), [1, embeddings_normalized.shape[1]])
    
    # Compute the contrastive loss
    margin = 1.0
    contrastive_loss = tf.reduce_mean(y_true_expanded * tf.square(embeddings_normalized - embeddings_projected) +
                                      (1 - y_true_expanded) * tf.square(tf.maximum(margin - tf.norm(embeddings_normalized - embeddings_projected, axis=-1), 0)))
    return contrastive_loss



# Load and preprocess the data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = y_train.flatten()
y_test = y_test.flatten()

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Prepare the labels for contrastive learning
labels_train_contrastive = tf.concat([tf.ones_like(y_train, dtype=tf.float32),
                                      tf.zeros_like(y_train, dtype=tf.float32)], axis=0)
labels_val_contrastive = tf.concat([tf.ones_like(y_val, dtype=tf.float32),
                                    tf.zeros_like(y_val, dtype=tf.float32)], axis=0)

# Encode the labels for classification
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Define the contrastive learning optimizer
contrastive_optimizer = tf.keras.optimizers.Adam()

# Training loop for contrastive learning
num_epochs_contrastive = 10
batch_size_contrastive = 32
num_steps_contrastive = len(X_train) // batch_size_contrastive

for epoch in range(num_epochs_contrastive):
    epoch_loss = 0.0

    for step in range(num_steps_contrastive):
        start = step * batch_size_contrastive
        end = (step + 1) * batch_size_contrastive

        batch_images = X_train[start:end]
        batch_labels = labels_train_contrastive[start:end]

        with tf.GradientTape() as tape:
            embeddings_normalized = contrastive_model(batch_images, training=True)
            embeddings_projected = projection_model(batch_images, training=True)
            loss = contrastive_loss(batch_labels, embeddings_normalized, embeddings_projected)
        
        gradients = tape.gradient(loss, contrastive_model.trainable_variables + projection_model.trainable_variables)
        contrastive_optimizer.apply_gradients(zip(gradients, contrastive_model.trainable_variables + projection_model.trainable_variables))

        epoch_loss += loss.numpy()

    avg_loss = epoch_loss / num_steps_contrastive
    print(f"Contrastive Learning - Epoch {epoch+1}/{num_epochs_contrastive}, Loss: {avg_loss}")

# Freeze the encoder model
encoder_model.trainable = False

# Add a new classifier layer
output_layer = Dense(10, activation='softmax')(encoder_model.output)

# Define the new model for classification
classification_model = Model(encoder_model.input, output_layer)

# Compile the classification model
classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the classification model
classification_model.fit(X_train, y_train_encoded, epochs=10, validation_data=(X_val, y_val_encoded))

# Evaluate the performance of the classification model
test_loss, test_accuracy = classification_model.evaluate(X_test, y_test_encoded, verbose=2)
print(f"Classification Model - Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")